In [1]:
data_var = '2023-12-30'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9967,2023-12-30,Espanha Acb,14:00,Palencia,Bilbao,2.14,1.72,157.5,1.81,1.93,1.5,1.98,1.79,6DoAxGXh,0.467290,0.581395,0.552486,0.518135,0.048685,0.0,0.0,NaN,NaN,375.238,241.994027,0.644908,217.822,154.557691,0.709560,0.000,0.000,165.60,317.25,0.0,0.0,0.0,0.0,0.153878,0.045376,0.071273,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
9968,2023-12-30,Espanha Acb,16:45,Basquet Girona,Baskonia,3.27,1.35,168.5,1.87,1.92,5.5,2.07,1.70,zmjEyzma,0.305810,0.740741,0.534759,0.520833,0.046551,0.4,0.0,NaN,NaN,284.234,212.919045,0.749098,136.176,43.334094,0.318221,162.966,0.000,127.30,209.56,0.0,0.0,0.0,0.0,0.587725,0.018657,0.138795,0.10,0.020,113.500000,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
9969,2023-12-30,Espanha Acb,16:45,Gran Canaria,Breogan,1.14,5.70,158.5,1.81,1.95,-12.5,2.03,1.71,EwkIzf25,0.877193,0.175439,0.552486,0.512821,0.052632,0.6,0.0,NaN,NaN,168.968,100.006981,0.591869,243.436,164.674986,0.676461,112.774,0.000,102.09,237.12,0.0,0.0,0.0,0.0,0.942809,0.052657,0.121002,-0.15,-0.030,-4.666667,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
9970,2023-12-30,Eua Nba,00:00,Portland Trail Blazers,San Antonio Spurs,1.51,2.82,230.5,1.80,2.08,-5.5,1.97,1.97,xdEZv06f,0.662252,0.354610,0.555556,0.480769,0.016862,0.6,0.4,NaN,NaN,357.576,131.822908,0.368657,605.504,650.028220,1.073532,343.028,679.968,533.00,402.22,1.0,0.0,0.0,0.0,0.427857,0.102057,0.000000,2.54,0.508,1.003937,0.37886,0.3,-0.07886,-5.00,-1.000,-1.820000,0.213746,0.0,-0.213746
9971,2023-12-30,Eua Nba,00:30,Los Angeles Clippers,Memphis Grizzlies,1.60,2.57,227.5,1.80,2.08,-4.5,2.05,1.89,GnDwvKL0,0.625000,0.389105,0.555556,0.480769,0.014105,0.6,0.6,NaN,NaN,206.324,64.585039,0.313027,309.834,100.968852,0.325880,188.130,359.238,127.69,412.65,0.0,0.0,0.0,0.0,0.328966,0.102057,0.057430,1.01,0.202,2.970297,0.66758,0.7,0.03242,3.08,0.616,2.548701,0.327120,0.5,0.172880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10199,2023-12-30,Eua Ncaa,21:30,UTRGV,Incarnate Word,1.59,2.35,153.5,1.80,1.91,-4.5,2.00,1.73,dYbssQm6,0.628931,0.425532,0.555556,0.523560,0.054463,0.0,0.4,NaN,NaN,321.736,233.124920,0.724585,319.270,202.385936,0.633902,0.000,329.660,123.88,616.40,0.0,1.0,0.0,0.0,0.272792,0.041931,0.102369,0.00,0.000,inf,0.00000,0.0,0.00000,4.20,0.840,1.607143,0.000000,0.0,0.000000
10200,2023-12-30,Eua Ncaa,22:00,Grand Canyon,Louisiana Tech,1.31,3.38,138.5,1.80,1.91,-7.5,1.83,1.83,OChSZva2,0.763359,0.295858,0.555556,0.523560,0.059217,0.6,0.4,NaN,NaN,124.630,45.874315,0.368084,117.862,23.307186,0.197750,106.818,152.572,79.80,149.65,0.0,0.0,0.0,0.0,0.624184,0.041931,0.000000,0.50,0.100,3.100000,0.00000,0.0,0.00000,-1.48,-0.296,-8.040541,0.000000,0.0,0.000000
10201,2023-12-30,Eua Ncaa,22:00,UC Riverside,UC Santa Barbara,2.00,1.80,142.5,1.83,1.83,1.5,1.80,1.91,SvG5RDMI,0.500000,0.555556,0.546448,0.546448,0.055556,0.8,0.0,NaN,NaN,303.664,302.395612,0.995823,183.532,109.043974,0.594141,115.352,0.000,112.34,109.12,0.0,0.0,0.0,0.0,0.074432,0.000000,0.041931,0.86,0.172,5.813953,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
10202,2023-12-30,Eua Ncaa,23:00,New Mexico State,California Baptist,2.00,1.80,136.5,1.80,1.91,-1.5,2.00,1.73,4872I406,0.500000,0.555556,0.555556,0.523560,0.055556,0.0,0.0,NaN,NaN,436.440,

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Portland Trail Blazers,San Antonio Spurs,230.5,1.80,1.0000,Over
1,00:30,Eua Nba,Los Angeles Clippers,Memphis Grizzlies,227.5,1.80,1.0000,Over
2,22:00,Eua Nba,Chicago Bulls,Philadelphia 76ers,223.5,1.89,1.0000,Over
3,15:00,Eua Ncaa,Central Conn. St.,St. Elizabeth,141.5,1.91,0.5015,Over
4,15:00,Eua Ncaa,Citadel,Toccoa Falls,150.5,1.91,0.5150,Over
5,16:00,Eua Ncaa,Duquesne,Cleary University,139.5,1.91,0.5776,Over
